In [1]:
from ansys.mapdl.core import launch_mapdl
import pandas as pd
import numpy as np

In [2]:
from functions import kagome_horizontal

In [6]:
# Start an MAPDL instance
try:
    mapdl = launch_mapdl(start_instance=False, port=50052)
except:
    mapdl = launch_mapdl(start_instance=True, port=50052)

In [90]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Research     
 01078600  VERSION=LINUX x64     12:08:10  DEC 03, 2024 CP=    149.997

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [91]:
stress_strain = pd.read_excel('PLA stress-strain.xlsx')
stress_strain = stress_strain.iloc[:8]  # due to limitations of MISO model
stress_strain['y'] = stress_strain['y (MPa)']*1e6
stress_strain['x'] = stress_strain['x']/100
UTS = stress_strain['y'].max()
Youngs_Modulus = stress_strain['y'][1]/stress_strain['x'][1]
max_plastic_strain = stress_strain['x'].iloc[-1]

# mapdl.tb('PLASTIC', 1, '', 'MISO')
# mapdl.tbtemp()

mapdl.mp('EX', 1, Youngs_Modulus)   # Young's Modulus in Pascals
mapdl.mp('PRXY', 1, 0.33)   # Poisson's Ratio

# # Non linear 부분 정의
# stress_strain['x'] = stress_strain['x'] - stress_strain['y']/Youngs_Modulus
# for i in range(len(stress_strain)):
#     mapdl.tbpt(i, stress_strain['x'][i], stress_strain['y'][i])

mapdl.tb(lab='DMGI', tbopt=1)  # Failure Criterion을 만족하면 Damage Initiation. 왜 안되는지 모르겠음.

mapdl.tbdata(1, 2, 2, 2, 2)  # Maximum Stress Criterion

# Failure Criterion 정의.
# Failure Criterion을 바꾸려면 mapdl.fctyp. Default는 EMAX, SMAX, TWSI, TWSR.
mapdl.fc(mat=1, lab1='S', lab2='XTEN', data1=UTS)
mapdl.fc(mat=1, lab1='S', lab2='YTEN', data1=UTS)
mapdl.fc(mat=1, lab1='S', lab2='ZTEN', data1=UTS)

mapdl.tb(lab='DMGE', tbopt=1)  # Damage Evolution
mapdl.tbdata(1, 0.5, 0.5, 0.5, 0.5)

DATA FOR  DMGE  TABLE FOR MATERIAL   1 AT TEMPERATURE=  0.0000    
 LOC=  1 5.00000e-01     5.00000e-01     5.00000e-01     5.00000e-01

In [92]:
mapdl.fclist()

FCLIST output for all materials at   0.0 degrees.

 Stress Failure Criteria for Material    1  evaluated at   0.0 degrees
     XTEN      XCMP      YTEN      YCMP      ZTEN      ZCMP      XY        YZ        XZ  
  0.619E+08-0.619E+08 0.619E+08-0.619E+08 0.619E+08-0.619E+08 0.000E+00 0.000E+00 0.000E+00
     XYCP      YZCP      XZCP      XZIT      XZIC      YZIT      YZIC      G1G2      ETAL
 -0.100E+01-0.100E+01-0.100E+01 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00 0.000E+00
     ETAT      ALP0
  0.000E+00 0.530E+02

 *** NOTE ***                            CP =     150.202   TIME= 12:08:12
 The strain failure criteria input for one or more materials (such as    
 Material 1) was not provided.

In [93]:
mapdl.tblist('DMGI')

LIST DATA TABLE  DMGI  FOR MATERIAL      1

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Research     
 01078600  VERSION=LINUX x64     12:08:12  DEC 03, 2024 CP=    150.204

                                                                               



               DMG INIT (DMGI) Table For Material     1
               Failuare Criteria Type                                                          

                     1 
 Temps     0.0000000e+00 
 FIBR TEN 2.0000000e+00 
 FIBR CMP 2.0000000e+00 
 MATX TEN 2.0000000e+00 
 MATX CMP 2.0000000e+00

In [94]:
n_cell = 3
relative_density = 0.45
three_dim = False
external_wall = False

In [95]:
wall_thickness = kagome_horizontal(mapdl, n_cell, relative_density, three_dim=three_dim, external_wall=external_wall)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import traceback
import os

design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000

mesh_size = 1.0
lattice_type = 'kagome_h'
filename='temp'
stop_when_fail=False
total_displacement=20/1000

line_design_part = mapdl.lsel("S", "LOC", "X", 0, design_width)
mapdl.lsel("NONE")
for line in line_design_part:
    mapdl.lsel("A", "LINE", vmin=line, vmax=line)
mapdl.lesize("ALL", wall_thickness*mesh_size, kforc=1)  # 격자 부분의 mesh를 더 작게 함
mapdl.lsel("ALL")

# Define default mesh size
if three_dim:
    mapdl.esize(design_height/3)
else:
    mapdl.esize(design_height/6)

mapdl.mopt("EXPND", 0.7)  # Decrease the area mesh expansion.
mapdl.et(1, 'PLANE')

if three_dim:
    mapdl.et(1, 'SOLID187')
    mapdl.vmesh("all")
else:
    # define a PLANE183 element type with thickness
    mapdl.et(1, "PLANE183", kop1=1, kop3=3)
    mapdl.r(1, total_z)  # thickness of 5mm
    # Mesh the volume
    mapdl.amesh("all")
print("Preprocessing Finished")


# Solution
mapdl.solution()
mapdl.antype('STATIC')
# mapdl.nlgeom('ON')  # Enable large deformation effects

total_displacement = total_displacement  # 20 mm total displacement
number_of_steps = int(total_displacement/(0.1/1000))  # 0.1mm 간격
displacement_increment = total_displacement / number_of_steps

# mapdl.nropt('UNSYM')  # Use unsymmetric matrix if convergence issues occur
mapdl.autots('ON')    # Enable automatic time stepping
# mapdl.nliter(50, 0, 0, 0, 0)  # Set maximum number of equilibrium iterations
# mapdl.cnvtol('F', 0.1)  # Force convergence tolerance
# mapdl.nsubst(20, 100, 1)
# mapdl.deltim(dtime = "AUTOTS", dtmin = 0, dtmax = 1)

# Initialize arrays to store results
reaction_forces = []
applied_displacements = []

for step in range(1, number_of_steps + 1):
    time_start = time.time()
    current_disp = displacement_increment * step
    mapdl.ddele('ALL', 'UX')  # Remove previous displacement BC
    mapdl.nsel('S', 'LOC', 'X', (total_width + design_width)/2)
    mapdl.d('ALL', 'UX', current_disp)
    mapdl.d("ALL", "UY", 0)
    if three_dim:
        mapdl.d("ALL", "UZ", 0)
    mapdl.nsel('S', 'LOC', 'X', (-total_width + design_width)/2)
    mapdl.d("ALL", "ALL", 0)
    mapdl.nsel('ALL')
    
    time_solutionstart = time.time()
    # Solve current load step
    try:
        mapdl.solve()
    except:
        with open(f"result/{lattice_type}_{'wall' if external_wall else 'nowall'}/error_log.txt", "a") as file:
            file.write(f"An error occurred at mesh size:{mesh_size}, iteration:{step}, filename: {filename}\n")
            file.write(traceback.format_exc())  # Writes the full stack trace
            file.write("\n")
        print("Solution Error!\n\n")
        try:
            mapdl = launch_mapdl(start_instance=False, port=50052)
        except:
            mapdl = launch_mapdl(start_instance=True, port=50052)
            pass        
    time_solutionend = time.time()
    # print(f"Solution time = {int(time_solutionend-time_solutionstart):4}s")

    # Post-process to check for failure criterion
    mapdl.post1()
    mapdl.set('LAST')
    
    # Get nodal principal stress
    mapdl.nsel('ALL')
    
    principal_stress = np.maximum.reduce([mapdl.post_processing.nodal_principal_stress("1"),
                mapdl.post_processing.nodal_principal_stress("2"),
                mapdl.post_processing.nodal_principal_stress("3")])


    mapdl.nsel('ALL')
    mapdl.nsel('S', 'LOC', 'X', (-total_width + design_width)/2)

    reaction_forces_selected = mapdl.prrsol('FX').to_dataframe(columns=["NODE", "FX"])
    reaction_force = abs(reaction_forces_selected['FX'].sum())

    # Get the reaction forces from the result object
    # result = mapdl.result
    # # 이거를 result 안쓰고 하는 법 없나? result 문법을 잘 모르겠음
    # nnum, reaction_force = result.nodal_static_forces(0)
    # mapdl.nsel('ALL')
    # mapdl.nsel('S', 'LOC', 'X', 0)
    # selected_nodes = mapdl.mesh.nnum  # Node numbers in the current selection
    # reaction_force = reaction_force[selected_nodes-1].sum(axis=0)[0]
    
    reaction_forces.append(reaction_force)
    applied_displacements.append(current_disp)

    print(f"Step{step:3} completed.  Current displacement : {current_disp*1000:5.2f}  Elapsed Time : {int(time.time()-time_start):4}s")

    # Principal stress / UTS

    if stop_when_fail:
        if (principal_stress > UTS).sum() > 0:
            print("Specimen failed!\n\n")
            break

    mapdl.solution()

current_dir = os.getcwd()
df = pd.DataFrame({'displacement': applied_displacements, 'force': reaction_forces})
df.to_csv(f'{current_dir}/result/{lattice_type}_{"wall" if external_wall else "nowall"}/stress_strain_{filename}_{mesh_size:1.2f}'.replace(".", "_")+'.csv')
mapdl.save(fname=f'{current_dir}/result/{lattice_type}_{"wall" if external_wall else "nowall"}/db_{filename}_{mesh_size:1.2f}'.replace(".", "_"), ext="db")
mapdl.reswrite(fname = f'{current_dir}/result/{lattice_type}_{"wall" if external_wall else "nowall"}/res_{filename}_{mesh_size:1.2f}'.replace(".", "_"))

mapdl.finish()

Preprocessing Finished


In [28]:
from functions import kagome_sine_horizontal, triangle_horizontal, kagome_horizontal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import traceback
import os

In [5]:
def lattice_tensile_test(mapdl, lattice_type, mesh_size, n_cell, relative_density, amplitude_ratio=0.5, three_dim=False, filename='', external_wall=True):
    mapdl.clear()
    mapdl.prep7()

    lattice_type_list = ['kagome_h', 'kagome_v', 'kagome_sine_h', 'kagome_sine_v', 'triangle_h', 'triangle_v']
    if not lattice_type in lattice_type_list:
        raise Exception("Invalid lattice type")

    print(f"{'3D' if three_dim else '2D'} {lattice_type} Simulation Start. Mesh Size: {mesh_size}")
    print(f"Name: {filename}")
    design_width = 60/1000
    design_height = 25/1000
    total_width = 144/1000
    total_z = 5/1000
    if lattice_type == 'kagome_h':
        wall_thickness = kagome_horizontal(mapdl, n_cell, relative_density, three_dim=three_dim, external_wall=external_wall)
    elif lattice_type == 'kagome_v':
        pass
    elif lattice_type == 'kagome_sine_h':
        try:
            wall_thickness = kagome_sine_horizontal(mapdl, n_cell, relative_density, amplitude_ratio, three_dim=three_dim, external_wall=external_wall)
        except:
            with open(f"result/{lattice_type}_{'wall' if external_wall else 'nowall'}/error_log.txt", "a") as file:
                file.write(f"An error occurred at mesh size:{mesh_size}, filename: {filename}\n")
                file.write(traceback.format_exc())  # Writes the full stack trace
                file.write("\n")
            print("Geometry Error!\n\n")
            return
    elif lattice_type == 'kagome_sine_v':
        pass
    elif lattice_type == 'triangle_h':
        wall_thickness = triangle_horizontal(mapdl, n_cell, relative_density, three_dim=three_dim, external_wall=external_wall)
    elif lattice_type == 'triangle_v':
        pass

    # 물성치 입력
    stress_strain = pd.read_excel('PLA stress-strain.xlsx')
    stress_strain = stress_strain.iloc[:8]  # due to limitations of MISO model
    stress_strain['y'] = stress_strain['y (MPa)']*1e6
    stress_strain['x'] = stress_strain['x']/100
    UTS = stress_strain['y'].max()
    Youngs_Modulus = stress_strain['y'][1]/stress_strain['x'][1]
    max_plastic_strain = stress_strain['x'].iloc[-1]

    mapdl.mp('EX', 1, Youngs_Modulus)   # Young's Modulus in Pascals
    mapdl.mp('PRXY', 1, 0.33)   # Poisson's Ratio

    # Non linear 부분 정의
    stress_strain['x'] = stress_strain['x'] - stress_strain['y']/Youngs_Modulus
    mapdl.tb('PLASTIC', 1, '', 'MISO')
    for i in range(len(stress_strain)):
        mapdl.tbpt(i, stress_strain['x'][i], stress_strain['y'][i])

    # Meshing
    line_design_part = mapdl.lsel("S", "LOC", "X", 0, design_width)
    mapdl.lsel("NONE")
    for line in line_design_part:
        mapdl.lsel("A", "LINE", vmin=line, vmax=line)
    mapdl.lesize("ALL", wall_thickness*mesh_size, kforc=1)  # 격자 부분의 mesh를 더 작게 함
    mapdl.lsel("ALL")

    # Define default mesh size
    if three_dim:
        mapdl.esize(design_height/3)
    else:
        mapdl.esize(design_height/6)

    mapdl.mopt("EXPND", 0.7)  # Decrease the area mesh expansion.
    mapdl.et(1, 'PLANE')

    if three_dim:
        mapdl.et(1, 'SOLID187')
        mapdl.vmesh("all")
    else:
        # define a PLANE183 element type with thickness
        mapdl.et(1, "PLANE183", kop1=1, kop3=3)
        mapdl.r(1, total_z)  # thickness of 5mm
        # Mesh the volume
        mapdl.amesh("all")
    print("Preprocessing Finished")


    # Solution
    mapdl.solution()
    mapdl.antype('STATIC')
    # mapdl.nlgeom('ON')  # Enable large deformation effects

    total_displacement = 5/1000  # 20 mm total displacement
    number_of_steps = 50
    displacement_increment = total_displacement / number_of_steps

    # mapdl.nropt('UNSYM')  # Use unsymmetric matrix if convergence issues occur
    mapdl.autots('ON')    # Enable automatic time stepping
    # mapdl.nliter(50, 0, 0, 0, 0)  # Set maximum number of equilibrium iterations
    # mapdl.cnvtol('F', 0.1)  # Force convergence tolerance
    # mapdl.nsubst(20, 100, 1)
    # mapdl.deltim(dtime = "AUTOTS", dtmin = 0, dtmax = 1)

    # Initialize arrays to store results
    reaction_forces = []
    applied_displacements = []

    for step in range(1, number_of_steps + 1):
        time_start = time.time()
        current_disp = displacement_increment * step
        mapdl.ddele('ALL', 'UX')  # Remove previous displacement BC
        mapdl.nsel('S', 'LOC', 'X', (total_width + design_width)/2)
        mapdl.d('ALL', 'UX', current_disp)
        mapdl.d("ALL", "UY", 0)
        if three_dim:
            mapdl.d("ALL", "UZ", 0)
        mapdl.nsel('S', 'LOC', 'X', (-total_width + design_width)/2)
        mapdl.d("ALL", "ALL", 0)
        mapdl.nsel('ALL')
        
        time_solutionstart = time.time()
        # Solve current load step
        try:
            mapdl.solve()
        except:
            with open(f"result/{lattice_type}_{'wall' if external_wall else 'nowall'}/error_log.txt", "a") as file:
                file.write(f"An error occurred at mesh size:{mesh_size}, iteration:{step}, filename: {filename}\n")
                file.write(traceback.format_exc())  # Writes the full stack trace
                file.write("\n")
            print("Solution Error!\n\n")
            try:
                mapdl = launch_mapdl(start_instance=False, port=50052)
            except:
                mapdl = launch_mapdl(start_instance=True, port=50052)
                return        
        time_solutionend = time.time()
        # print(f"Solution time = {int(time_solutionend-time_solutionstart):4}s")

        # Post-process to check for failure criterion
        mapdl.post1()
        mapdl.set('LAST')
        
        # Get nodal principal stress
        mapdl.nsel('ALL')
        
        # principal_stress = np.maximum.reduce([mapdl.post_processing.nodal_principal_stress("1"),
        #             mapdl.post_processing.nodal_principal_stress("2"),
        #             mapdl.post_processing.nodal_principal_stress("3")])


        mapdl.nsel('ALL')
        mapdl.nsel('S', 'LOC', 'X', (-total_width + design_width)/2)

        reaction_forces_selected = mapdl.prrsol('FX').to_dataframe(columns=["NODE", "FX"])
        reaction_force = abs(reaction_forces_selected['FX'].sum())

        # Get the reaction forces from the result object
        # result = mapdl.result
        # # 이거를 result 안쓰고 하는 법 없나? result 문법을 잘 모르겠음
        # nnum, reaction_force = result.nodal_static_forces(0)
        # mapdl.nsel('ALL')
        # mapdl.nsel('S', 'LOC', 'X', 0)
        # selected_nodes = mapdl.mesh.nnum  # Node numbers in the current selection
        # reaction_force = reaction_force[selected_nodes-1].sum(axis=0)[0]
        
        reaction_forces.append(reaction_force)
        applied_displacements.append(current_disp)

        print(f"Step{step:3} completed.  Current displacement : {current_disp*1000:5.2f}  Elapsed Time : {int(time.time()-time_start):4}s")
        print(f"{current_disp:.2f}\t{reaction_force:.1f}")
        # Principal stress / UTS

        # if (principal_stress > UTS).sum() > 0:
        #     print("Specimen failed!\n\n")
        #     break

        mapdl.solution()

    current_dir = os.getcwd()
    df = pd.DataFrame({'displacement': applied_displacements, 'force': reaction_forces})
    # df.to_csv(f'{current_dir}/result/{lattice_type}_{"wall" if external_wall else "nowall"}/stress_strain_{filename}_{mesh_size:1.2f}'.replace(".", "_")+'.csv')
    # mapdl.save(fname=f'{current_dir}/result/{lattice_type}_{"wall" if external_wall else "nowall"}/db_{filename}_{mesh_size:1.2f}'.replace(".", "_"), ext="db")
    # mapdl.reswrite(fname = f'{current_dir}/result/{lattice_type}_{"wall" if external_wall else "nowall"}/res_{filename}_{mesh_size:1.2f}'.replace(".", "_"))

    mapdl.finish()

    return df

In [6]:
n_cell = 3
relative_density = 0.45

df = lattice_tensile_test(mapdl, lattice_type='kagome_sine_h', mesh_size=0.5, n_cell=n_cell, relative_density=relative_density,
                             amplitude_ratio=0.5, three_dim=False, filename=f'n{n_cell}_r{int(relative_density*100)}', external_wall=False)
df

2D kagome_sine_h Simulation Start. Mesh Size: 0.5
Name: n3_r45
Preprocessing Finished
Step  1 completed.  Current displacement :  0.10  Elapsed Time :   10s
0.00	42.1
Step  2 completed.  Current displacement :  0.20  Elapsed Time :   13s
0.00	84.2
Step  3 completed.  Current displacement :  0.30  Elapsed Time :   15s
0.00	126.3
Step  4 completed.  Current displacement :  0.40  Elapsed Time :   16s
0.00	168.3
Step  5 completed.  Current displacement :  0.50  Elapsed Time :   16s
0.00	210.1
Step  6 completed.  Current displacement :  0.60  Elapsed Time :   18s
0.00	251.5
Step  7 completed.  Current displacement :  0.70  Elapsed Time :   36s
0.00	292.2
Step  8 completed.  Current displacement :  0.80  Elapsed Time :   50s
0.00	332.2
Step  9 completed.  Current displacement :  0.90  Elapsed Time :   66s
0.00	371.2
Step 10 completed.  Current displacement :  1.00  Elapsed Time :   63s
0.00	409.1
Step 11 completed.  Current displacement :  1.10  Elapsed Time :   85s
0.00	445.7
Step 12 comple

,displacement,force
0,0.0001,42.1180
1,0.0002,84.2347
2,0.0003,126.3347
3,0.0004,168.3306
4,0.0005,210.1023
5,0.0006,251.4730
6,0.0007,292.2030
7,0.0008,332.1940
8,0.0009,371.2000
9,0.0010,409.0740


In [9]:
import os

current_dir = os.getcwd()

mapdl.post1()

mapdl.save(fname=f"{current_dir}/result/kagome_sine_h_nowall/res_n3_r45_0_30_limitless", ext="db")
# mapdl.run(f"FILE, {current_dir}/result/result_iteration, rst")
mapdl.reswrite(fname = f"{current_dir}/result/kagome_sine_h_nowall/res_n3_r45_0_30_limitless")

MapdlCommandIgnoredError: *** ERROR ***                           CP =   15988.052   TIME= 10:51:23
 No data stored in database.                                             
  The RESWRITE command is ignored.

Ignore these messages by setting 'ignore_errors'=True.


In [10]:
df.to_csv(f"{current_dir}/result/kagome_sine_h_nowall/stress_strain_n3_r45_0_30_limitless")